<a href="https://colab.research.google.com/github/ElaYJ/Study_Deep_Learning/blob/main/Lecture/52_GPT_HF_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train

- Fine Tuning 미세 조정 과정

    - Loss Function 생성

    - Optimizer 만들기

    - backward를 통해 weights를 업데이트

- 실무에서 구현 가능한 예제

### - tokenizer

- 간단한 tokenizer 구현

- 모델이 사용한 tokenizer를 쓰지 않으면 성능이 많이 떨어진다.

In [1]:
# https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english#model-details

# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [2]:
from transformers import AutoTokenizer

In [3]:
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
raw_inputs = [
    "I've been waiting for a huggingface course my whole life",
    "i hate this so much!",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt') # 'pt' - pytorch

In [5]:
inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [6]:
inputs['input_ids']
#--> 101은 시작을 나타내는 token
#--> 1045은 'I'를 나타내는 token
#--> 0으로 padding이 됨.

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]])

In [7]:
inputs['attention_mask']
#--> 문장의 길이가 다르므로 input 차원을 맞추기 위해 의미없는 0으로 padding 되어야 한다.

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])

In [8]:
from transformers import AutoModel

# 모델 불러오기
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [9]:
outputs = model(**inputs)
outputs

BaseModelOutput(last_hidden_state=tensor([[[-0.1378,  0.1974,  0.6768,  ..., -0.2884,  0.4254,  0.1880],
         [ 0.2657,  0.5706,  0.3957,  ..., -0.1252,  0.4597,  0.1770],
         [ 0.7825,  0.1824,  0.2876,  ...,  0.2613, -0.0953, -0.5923],
         ...,
         [ 0.1757,  0.3456,  0.3675,  ..., -0.5196,  0.6726, -0.2503],
         [ 0.0929,  0.4920,  0.3286,  ..., -0.3443,  0.4261, -0.0208],
         [ 0.2093,  0.4148,  0.4770,  ...,  0.3803,  0.4811, -0.5183]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.2319,  0.8268, -0.0312,  ..., -0.0764, -0.8509, -0.1043],
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [10]:
outputs.last_hidden_state.shape

torch.Size([2, 15, 768])

- Model hidden output

    - ForCasulLM

    - FormaskedLM

    - ForQuestionAnswering

    - etc.

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [12]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [13]:
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.4683,  1.5105],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
import torch

prediction = torch.nn.functional.softmax(outputs.logits, dim=1)
prediction

tensor([[4.8393e-02, 9.5161e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)

- BERT

In [15]:
from transformers import BertConfig, BertModel

config = BertConfig() # default 생성
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [16]:
model = BertModel(config)
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [17]:
config.hidden_size

768

In [18]:
# config 값을 조정해 fine tuning 할 수 있다.

config.hidden_size = 48

model = BertModel(config)
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 48, padding_idx=0)
    (position_embeddings): Embedding(512, 48)
    (token_type_embeddings): Embedding(2, 48)
    (LayerNorm): LayerNorm((48,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=48, out_features=48, bias=True)
            (key): Linear(in_features=48, out_features=48, bias=True)
            (value): Linear(in_features=48, out_features=48, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=48, out_features=48, bias=True)
            (LayerNorm): LayerNorm((48,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
   

In [19]:
# train...
# 학습한 후 모델을 저장

model.save_pretrained("./test")

- tokenizer

    - Split on spaces : 공백을 기반으로 토큰화

    - Split on punctuation : 구두점을 기반으로 토큰화

In [20]:
# 공백 기반 토큰화 예제

tokenized_text = "Jim Henson was puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'puppeteer']


- 토큰이 중요한 이유

    - '[UNK]' : unknown

    - unknown token이 많을수록 모델을 fine tuning 할 때 애로사항이 많이 생길 수 있다.

    - 그래서 unknown token을 어떻게 없앨 수 있을까가 해결해야 할 큰 숙제이다.

    - 예를 들어, 일반적인 책이나 wiki pedia를 데이터셋으로 학습한 모델에 twitter를 input data로 사용할 때 트위터에 많은 '#~'를 텍스트로 처리할 것인지, 아니면 지울 것인지, 어떻게 처리할 것인지 결정해야 한다. 이모티콘도 string올 표현되는 바이트이므로 어떻게 처리할 지에 대해 고민해야한다. 너무 많은 unknown token이 존재하게 되면 제대로된 evaluation이 불가능하게 된다.

- BERT는 subword(접미사, 접두어, 등)으로 tokenize 된다.

In [21]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens) #--> '##former' 이런 것들이 존재하게 된다.

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [22]:
# 만약 전자기기 메뉴얼 데이터셋이라면 BERT와 같이 tokenize 되는 모델은 맞지 않는 것이 된다.

sequence = "Using a Transformer network is menual, KT-12327"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'menu', '##al', ',', 'K', '##T', '-', '123', '##27']


In [23]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 13171, 1348, 117, 148, 1942, 118, 13414, 24458]


In [24]:
# 인덱스로 디코더도 가능하다.

tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 13171, 1348, 117, 148, 1942, 118, 13414, 24458])

'Using a Transformer network is menual, KT - 12327'

<br></br>

### - dataloader

- data를 load할 수 있으면 train(학습)도 가능하다.

- dataload --> model --> optimizer --> loss func. --> training

In [57]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
model(input_ids) #--> This line will fail. why? Shape이 맞지 않게 때문이다. --> Matix로 만들어 줘야 한다.

IndexError: too many indices for tensor of dimension 1

In [26]:
input_ids, input_ids.shape

(tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]),
 torch.Size([14]))

In [27]:
tokenized_inputs = tokenizer(sequence, return_tensors='pt')
print(tokenized_inputs['input_ids']) #--> 이렇게 하면 자동으로 행과 열의 형태로 만들어준다.

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [28]:
tokenized_inputs['input_ids'].shape
#--> batch 1개와 16개의 token
#--> 이런 식으로 batch 기반의 형태로 만들어 줘야 한다.
#--> tokenizer() 함수를 이용하면 이러한 행렬 형태로 만들어 주는데,
# tokens = tokenizer.tokenize(sequence)
# ids = tokenizer.convert_tokens_to_ids(tokens)
#--> 이런 식으로 따로 해주면 리스트 형이 만들어 진다.

torch.Size([1, 16])

In [29]:
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits) #--> 0과 1의 확률 --> 긍정

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [30]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [31]:
print(tokens)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']


- 한 문장이 아닌 여러 개의 문장일 때는 어떤 shape으로 input해줘야 할까?

In [32]:
# batch size가 2이고 문장의 길이가 3인 데이터셋을 넣었을 때 어떻게 될 것인가??

batched_ids = [
    [200,200,200],
    [200,200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0]
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [33]:
tokenizer.decode([200, tokenizer.pad_token_id]) #--> 200은 안쓰는 ID이다.

'[unused195] [PAD]'

- dataloader 구현

    - install한 datasets에서 가능하다.

    - https://wikidocs.net/166801

In [34]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


In [35]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [36]:
# 데이터셋 불러오기

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [37]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [38]:
raw_datasets['train']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [39]:
raw_datasets['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

- 데이터 전처리 과정의 자동화

In [40]:
# 이 raw_datasets을 맵핑해준다.
# 먼저 tokenize_function()을 정의해서 'sentence1', 'sentence2'라는 입력 피쳐를 맵핑시켜준다.

def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [41]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [42]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [43]:
# batch에 대한 것만 있고 padding에 대한 정보가 없으므로 정의해줄 필요가 있다.
# collator : 대조자

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#--> data_collator가 데이터셋에 대한 padding을 자동으로 해준다.

In [44]:
# 필요없는 컬럼을 삭제하고 컬럼명을 변경한다.

tokenized_datasets = tokenized_datasets.remove_columns(['sentence1','sentence2','idx'])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [45]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [46]:
from torch.utils.data import DataLoader

# 최종 데이터 로더
train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator
)

In [47]:
# 데이터 확인

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 72]),
 'token_type_ids': torch.Size([8, 72]),
 'attention_mask': torch.Size([8, 72])}

In [48]:
{k: print(v) for k, v in batch.items()}

tensor([0, 1, 0, 1, 1, 1, 1, 0])
tensor([[  101,  1996,  5229,  2020,  2579,  2000,  8323,  1999,  7625,  2103,
          1998,  3996,  1010,  1037,  2267,  2237,  2055,  2382,  2661,  2000,
          1996,  2167,  1012,   102,  1996,  5229,  2020,  2579,  2000,  8323,
          1999,  7625,  2103,  1998,  3996,  1010,  2055,  4466, 22287,  2167,
          1025,  2037,  3785,  2020,  2025,  3202,  2800,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1999,  2804,  2000,  1051,  1005,  6720,  1010,  2128,  7295,
         18331,  1005,  1055,  3484,  5448,  2001,  2587,  2011, 19867,  2585,
          2061, 19901,  1010,  7920,  2919,  2121, 18353,  9695,  1010,  1998,
          4459,  7987, 20211,  1012,   102,  5241,  2032,  1999,  1996,  3484,
          5448,  2020, 19867, 12834,  2154,  1051,  1005,  6720,  1010,  2

{'labels': None,
 'input_ids': None,
 'token_type_ids': None,
 'attention_mask': None}

<br></br>

### - pretrained model

In [49]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # Negative, Positive로 label 2개

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<br></br>

### - optimizer

In [50]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5, no_deprecation_warning=True)
# FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version.
# Use the PyTorch implementation torch.optim.AdamW instead,
# or set `no_deprecation_warning=True` to disable this warning

<br></br>

### - epoch

In [51]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
print(num_training_steps)

1377


<br></br>

### - device

- GPU cuda or CPU

In [52]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

<br></br>

### - training

- 학습 코드

In [53]:
from tqdm import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        data = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**data)

        loss = outputs.loss #--> loss가 model에 encapsulation되어 있다.
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)

100%|█████████▉| 1376/1377 [02:45<00:00,  7.84it/s]

<br></br>

### - evaluation

In [56]:
from datasets import load_metric

metric = load_metric("glue", "mrpc", trust_remote_code=True)
# FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets.
# Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
# FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric.
# You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
# You can avoid this message in future by passing the argument `trust_remote_code=True`.
# Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.

model.eval()
for batch in eval_dataloader:
    data = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**data)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.8455882352941176, 'f1': 0.8919382504288165}

-----

<br></br>

# Debugging

In [55]:
import transformers

transformers.__path__

['/usr/local/lib/python3.10/dist-packages/transformers']